# Scraping the Hollis library

In [73]:
import time
import requests
import bs4
import re
import xml
import pandas as pd
import numpy as np
import xmltodict
from unidecode import unidecode
from collections import OrderedDict

In [2]:
df = pd.read_excel('Bibliothek Philipp Jaffés - Final 14.10.17.xlsx', header=1)

In [3]:
base_url = 'http://webservices.lib.harvard.edu/rest/v2/hollisplus/search/dc/?q='
headers={'User-Agent': 'Mozilla/5.0'}

Helper functions

In [4]:
def reshaper(title):
    return '%27' + re.sub(' ', '%20', title) + '%27'

In [121]:
def check_match(title, potential_match):
    clean_title = unidecode(title).lower()
    if unidecode(potential_match).lower() == clean_title or unidecode(potential_match).lower() in clean_title \
      or clean_title in unidecode(potential_match).lower():
        return True
    return False

Main function. Contains lots of annoying little gotchas.

In [255]:
def check_hollis(title):
    if type(title) != str:
        print('Title missing')
        return
    url = base_url + reshaper(title)
    req = requests.get(url, headers=headers)
    try:
        results = xmltodict.parse(req.text)['results']['resultSet']['item']
    except TypeError:
        print("0 results found")
        return
    if type(results) == OrderedDict:
        if check_match(title, results['dc:title']):
            return results
        else:
            print("Found results, but none appear relevant: {}".format(results['dc:title']))
            return
    else:
        flattened = [i['dc:title'] if type(i['dc:title']) == list else [i['dc:title']] for i in results] # deal with lists
        for idx, a in enumerate(flattened):
            for result in a:
                if check_match(title, result):
                    return results[idx]
        print("Found results, but none appear relevant: {}".format(flattened))
    return

Empty dictionary, for when no results found

In [7]:
empty = OrderedDict({'@id':[None],
         '@position':[None],
         '@rank':[None],
         'cataloglink':[None],
         'dc:title':[None],
         'dc:type':[None],
         'authorlink':[None],
         'dc:creator':[None],
         'dc:publisher':[None],
         'dc:subject':[None],
         'dc:description':[None],
         'dc:date':[None],
         'dc:identifier':[None],
         'links':[None]} 
)

Go through everything and hit the Hollis library. Takes a few minutes.

In [167]:
results = []
for index,i in enumerate(df.Titel):
    print('Book number {}'.format(index + 1), i)
    results.append((index, check_hollis(i)))

Book number 1 Leben und denckwürdige Thaten Herrn Richards, erwählten Römischen Kaisers
0 results found
Book number 2 Urkunden der Stadt Brieg (Codex diplomaticus Silesiae, 9. Band)
Found results, but none appear relevant: [['III. Staat und Kirche in Schlesien vor der preußischen Besitzergreifung'], ['IV. Verbrechen in Krakau (1361-1405) und seiner Beistadt Kasimir (1370-1402)'], ['Johann von Neumarkts kirchliche Laufbahn'], ['Rezensionen'], ['Bibliographie zur Geschichte des Mittelalters aus slawischen Zeitschriften'], ['Bibliographie zur Geschichte des Mittelalters aus slawischen Zeitschriften, 2. Folge, I. Polen 1952-1957'], ['Urkundenpublikation und Urkundenforschung in Schlesien. Der Weg zum schlesischen Urkundenbuch'], ['Die schlesische Siedlungsformenforschung'], ['Urkundenpublikation und Urkundenforschung in Schlesien: Der Weg zum Schlesischen Urkundenbuch'], ['Spitäler in der Kirchenprovinz Gnesen im Mittelalter'], ['Literaturbericht'], ['Die Entwicklung von mhd.î - û - iu im 

Book number 20 Volumen primum complectens scriptores rerum episcopatus Bambergensis
Found results, but none appear relevant: [['Volumen primum [-secundum complectens scriptores rerum episcopatus Bambergensis'], ['XXII. INDEX BIBLIOGRAPHICUS.'], ['Oct. VII, Prooemium de Ratione Universa Operis'], ['Nov. II Pars I, Dies 3, S. Pirminius, abbas in Suevia, Commentarius Praevius']]
Book number 21 Geschichte des Deutschen Reiches unter Conrad dem Dritten 
Book number 22 Geschichte des Deutschen Reiches unter Lothar dem Sachsen
Book number 23 Regesta Pontificum Romanorum
Book number 24 nan
Title missing
Book number 25 Burkhard Stickel's Tagebuch, 1566-1598
Book number 26 Über die Echtheit und Bedeutung der Urkunde König Rudolf's I. betreffend die baierische Kur
0 results found
Book number 27 Die deutsche Königswahl bis zur goldenen Bulle
Book number 28 Das neumärkische Landbuch Markgraf Ludwig's des Aelteren vom Jahre 1337
Found results, but none appear relevant: [['Neue Erkenntnisse zu den Ki

Book number 60 Beiträge zur Geschichte der falschen Dekretalen
Found results, but none appear relevant: [['Beitræge zur Geschichte der falschen Dekretalen, par H. Wasserschleben'], ['Beitraege zur Geschichte der falschen Dekretalen, par le docteur H. Wasserschleben (Book Review)'], ["Beitraege zur geschichte der falschen dekretalen. Matériaux pour l'histoire des fausses décrétales (Book Review)"], ['Gregory IV for Aldric of Le Mans (833): A Genuine or Spurious Decretal?'], ['I. Stufen pseudoisidorischer Verfälschung: Untersuchungen zum Konzilsteil der pseudoisidorischen Dekretalen'], ['Von der Hilfswissenschaft zur Grundwissenschaft. Über Gegenwart und Zukunft des Handwerks der Historiker'], ['Diplomatik'], ['Erinnerung und Vergessen Die Gegenwart stiftet die Einheit der Vergangenheit'], ['DISKUSSIONSBEITRÄGE:Schlußwort'], ['LITERATURBERICHT'], ['Rezensionen'], ['Buchbesprechungen'], ['Der lange Schatten eines schwachen Herrschers: Ludwig der Fromme, die Kaiserin Judith, Pseudoisidor u

Book number 73 Corpus scriptorum ecclesiasticorum Latinorum
Book number 74 nan
Title missing
Book number 75 Recueil général des formules usitée dans l'Empire des Francs du Ve au Xe siècle
Found results, but none appear relevant: [["Recueil général des formules usitées dans l'empire des Francs du Ve au Xe siècle"], ["Receuil général des Formules usitées dans l'empire des Francs de Ve az Xe siècle par Eugène de Rozière I, II. Paris, Aug. Durand."], ["Recueil général des formules usitées dans l'empire des Francs du Ve au Xe siècle. Partie 2 / par Eugène de Rozière"], ["Recueil général des formules usitées dans l'empire des Francs du Ve au Xe siècle. Partie 3 / par Eugène de Rozière"], ["Recueil général des formules usitées dans l'empire des Francs du Ve au Xe siècle. Partie 1 / par Eugène de Rozière"], ["Recueil général des formules usitées dans l'empire des Francs du Ve au Xe siècle (Book Review)"]]
Book number 76 nan
Title missing
Book number 77 nan
Title missing
Book number 78 Decr

Book number 90 Incerti autoris saec. XIII Chronicon Halberstadense
Found results, but none appear relevant: Incerti auctoris saec. XIII. Chronicon Halberstadense, inde ab a. 780. usque ad a. 1209: aus der halberstädtischen handschrift
Book number 91 De Sardinia insula
Book number 92 Vom Reichsfuerstenstande. Forschungen zur Geschichte der Reichsverfassung zunaechst im XII. und XIII. Jahrhundert
Book number 93 Kaiser Heinrich IV. und seine Söhne
Found results, but none appear relevant: [['Kaiser Heinrich IV, und seine söhne'], ['Expressivität und Provokation : Jean-Baptiste Carpeaux\' Bronzeplastik "Ugolino und seine Söhne" und der Einfluss der französischen Malerei der Romantik'], ['Konstantin und seine Söhne in Aquileia'], ['Der Prätorianerpräfekt Q. Marcius Turbo und seine Söhne'], ['Marcius Hortalus, nobilis iuvenis, und seine Söhne'], ['Marcius Hortalus, nobilis iuvenis, und seine Söhne'], ['Blasedow und seine Söhne'], ['Noah und seine Söhne, oder die Neueinteilung der Welt nach d

...

Book number 270 De ortu et incremento superioritatis territorialis in comitatu
0 results found
Book number 271 Chronici Recherbergensis disquisitio
0 results found
Book number 272 Erzbischof Konrad I von Salzburg
Found results, but none appear relevant: [['Erzbischof Konrad I. von Salzburg. Inaug.-diss, Jena'], ['Erzbischof Konrad I. von Salzburg, 1106-1147. [Mit Bild]'], ['Stefan Weinfurter, Salzburger Bistumsreform und Bischofspolitik im 12. Jahrhundert. Der Erzbischof Konrad I. von Salzburg (1106 — 1147) und die Regularkanoniker (Kölner historische Abhandlungen 24)'], ['Salzburger Bistumsreform und Bischofspolitik im 12. [i.e. zwölften] Jahrhundert : der Erzbischof Konrad I. von Salzburg (1106-1147) u.d. Regularkanoniker'], ['Konrad von Wittelsbach, cardinal, erzbischof von Mainz und von Salzburg, deutscher reichskanzler: Zur feier des siebenhundertjährigen jubiläums des hauses Wittelsbach'], ['K. Zeillinger, Erzbischof Konrad I. von Salzburg 1106-1147 (Book Review)'], ['St. Weinfur

Book number 278 Die deutschen Rechtsbücher des Mittelalters und ihre Handschriften 
Book number 279 De arte medica saeculi XII.
Book number 280 Wirtembergische Geschichte, Bd. I-III
0 results found
Book number 281 nan
Title missing
Book number 282 nan
Title missing
Book number 283 Anfangsgrunde der Erd- Völker- und Staatenkunde
0 results found
Book number 284 Die Cambridger Lieder
Book number 285 Regesta chronologico-diplomatico regum atque imperatorum Romanorum / Die Urkunden der Römischen Könige und Kaiser
0 results found
Book number 286 Kleiner Atlas der neueren Geographie 
Book number 287 Historisch-geographischer Atlas der alten Welt 
Book number 288 Genève. Monnaies inédites fabriqueés à Pomponesco, Bozzolo, etc. (Mémoires et documents publiés par la Société d'histoire et d'archéologie de Genève, Bd. XVI)
Found results, but none appear relevant: Statistique du Département du Léman, publiée d'après le manuscrit original et présentée par H. O. Pappe (= Mémoires et documents publiés

Book number 303 Tarraco und Seine Denkmäler (Hermes, vol. I)
Book number 304 Eine Mailänder Synode vom Jahre 863
Found results, but none appear relevant: [['II. Die Metropolitanverfassung des karolingischen Regnum Italiae'], ['Das königliche Frauenkloster San Salvatore/Santa Giulia in Brescia im Spiegel seiner Memorialüberlieferung'], ['Papst Nikolaus I. und die Absetzung des Erzbischofs Johann von Ravenna'], ['Sedulius Scottus und die Gedichte des Codex Bernensis 363'], ['Zur Rekonstruktion einer vornehmlich bußrechtlichen Handschrift aus Bobbio (Hs. Vat. lat. 5751 ff. l-54v + Hs. Mailand, Bibl. Ambr. G. 58 sup. ff. 41r-64v)'], ['Die Reihung der Zeugen in Urkunden des frühen Mittelalters'], ['Eine im Original erhaltene Propagandaschrift des Erzbischofs Gunthar von Köln (865)'], ['Neue Studien zum alten Dom und zum römischen Forum in Köln'], ['Zeitschriftenschau'], ['Regesta Karolorum. Zu einem neuen Projekt der Regesta Imperii. Mit Ausblicken auf Urkunden und Kanzlei Kaiser Ludwigs II

Book number 314 Die Herren von Reste (Schlesische Provenzialblätter, Bd. VIII.?)
0 results found
Book number 315 Eine Schaumünze der Lucrezia Borgia (Berliner  Blätter für Münzkunde III.?)
0 results found
Book number 316 nan
Title missing
Book number 317 Ueber den Ursprung der deutschen Literatur
Book number 318 Mémoire sur une lettre inédite adressée à la reine Blanche par un habitant de La Rochelle (Bibliothèque de l'école des chartes, Bd. 17)
Found results, but none appear relevant: Abū 'Alis Spuren am Berliner Tiergarten. Ein diplomatisches Unikat aus dem Kreuzfahrerkönigreich Jerusalem
Book number 319 Opera omnia
Book number 320 Phaedri Augusti liberti fabulae Aesopiae
Book number 321 D. Iunii Iuvenalis satirarum libri
Book number 322 Brutus
Book number 323 Umfang der Diocese Magdeburg
Found results, but none appear relevant: [['Bericht über die Tätigkeit der Pius-Stiftung für Papsturkundenforschung im Jahre 1996/97'], ['Bericht über die Tätigkeit der Pius-Stiftung für Papsturkund

Found results, but none appear relevant: [['KAISERTUM UND ABENDLAND IN OTTONISCHER UND FRÜHSALISCHER ZEIT'], ['Bibliographie'], ['Bibliographie'], ['PARTIE BIBLIOGRAPHIQUE'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['BIBLIOGRAPHIE'], ['Bibliographie'], ['Bibliographie'], ['BIBLIOGRAPHIE'], ['BIBLIOGRAPHIE'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie'], ['Bibliographie']]
Book number 333 Die Hallische Lehntafel (N. Mitthl. d. Thür. Sächs. Ver. XI)
0 results found
Book number 334 De primordiis Henrici Leonis
Book number 335 Deutschlands Geschichtsquellen im Mittelalter
Book number 336 Geschichte des Deutschen Reiches unter Lothar dem Sachsen


Clean up into a dataframe, replacing an empty dict where necessary

In [169]:
swapped = []
for _,value in enumerate(results):
    if not value[1]:
        temp = empty.copy()
    else:
        temp = value[1]
    temp['index'] = _
    swapped.append(temp)

test = pd.DataFrame(swapped[0])
for i in swapped[1:]:
    test = test.append(i, ignore_index=True)

temp = [re.sub('@', '', i) for i in test.columns]
test.columns=[re.sub('dc:', '', i) for i in temp]
new = test[['index', 'title', 'creator', 'date', 'contributor', 'publisher', 'description','authorlink', 'cataloglink', 
            'links', 'notes', 'type', 'subject', 'identifier','findingAids']].copy()
new.set_index('index', inplace=True) 
new.sort_index(inplace=True)

Compare the new...

In [262]:
new.head()

,title,creator,date,contributor,publisher,description,authorlink,cataloglink,links,notes,type,subject,identifier,findingAids
index,,,,,,,,,,,,,,
0,[None],[None],[None],NaN,[None],[None],[None],[None],[None],NaN,[None],[None],[None],NaN
1,Ueber die chronik des Sulpicius Severus,"Bernays, Jacob",1861,NaN,Breslau: Grass,"['72, 1 (various pagings)', 'Jahresbericht d. ...","?limit=30&query=facet_creator,exact,Bernays++J...",NaN,NaN,NaN,book,"History, Ancient ; Severus, Sulpicius",HOLLIS+:,NaN
2,"[""XII schrifttafeln zu Wattenbach's Anleitung ...",NaN,1867,"Wattenbach, Wilhelm, 1819-1897. --Anleitung zu...","Leipzig, 1867.",NaN,NaN,"<a href=""http://id.lib.harvard.edu/alma/990072...",NaN,NaN,book,"Paleography, Greek.","['OCLC: 235973821', 'HOLLIS+: 9900728562602039...",NaN
3,Die Chronik des Cassiodorus Senator vom j. 519...,Cassiodorus,1864,"Mommsen, Theodor",Leipzig: S. Hirzel,"['[547]-696 p', '""Aus dem VIII. Bande der Abha...","?limit=30&query=facet_creator,exact,Cassiodorus",NaN,NaN,NaN,book,"Chronology, Roman ; Rome--History--Chronology",HOLLIS+:,NaN
4,Die Burg-kapelle zu Iben,"Peters, F",1869,NaN,Bonn: Gedr. auf Kosten des Vereins,"['10 p', 'illus., III pl. (1 fold.)']","?limit=30&query=facet_creator,exact,Peters++F",NaN,NaN,NaN,book,Iben (Castle),HOLLIS+:,NaN


and the old...

In [229]:
df.head()

,Autor,Titel,Ort,Jahr,Quelle,Signatur,Akzession Nr.,Bemerkungen
0,Ferdinand Piper,Vergleichender Kalender für 1869,Berlin,1869,NaN,3770 (1869),49469,Ohne Befund
1,Max Mueller,Ueber die Chronik des Sulpicius Severus,Berlin,1861,NaN,494 b 4°,50848,Ohne Befund
2,[Wilhelm Wattenbach],XII Schrifttafeln zu Wattenbach's Anleitung zu...,Leipzig,1867,NaN,3773 pb 4° (1867),50852,Ohne Befund
3,Theodor Mommsen,Die Chronik des Cassiodorus Senator,Leipzig,1861,NaN,4076 mc 4°,-,Leichte Korrektur
4,F. Peters,Die Burg-Kapelle zu Iben,Bonn,1869,NaN,3438 mV 4°,51985,Ohne Befund
